# Level 2: Simple Agentic 

This tutorial is designed to give new users an overview of how to use llama-stack's builtin agent framework effectively. It demonstrates basic single-tool usage and provides guidance on switching between a local development environment and a remote kubernetes cluster.

## Overview

This tutorial walks you through how to build your own AI agent who can search the web following these steps:
1. Connecting to a llama-stack server (remote or local).
2. Configuring an agent for tool use.
3. Running the agent.


## Prerequisites

Before starting, ensure you have the following:
- Access to a remote cluster or a local Podman setup.
- API keys and user variables configured (e.g., `inference_model`, `TAVILY_SEARCH_API_KEY`, `LLAMA_STACK_ENDPOINT`).
- A Tavily API key is required. You can register for one at [https://tavily.com/](https://tavily.com/).


## Building a Web Search Agent
### 1. Connecting to a llama-stack server.
#### 1.1 Setting Up the Environment
- Import the necessary libraries.
- Defining user variables.
- Connection options
    - **Remote Server**: Set `remote=True` and provide the remote llama-stack endpoint.
    - **Local Server**: Set `remote=False` and provide the local llama-stack endpoint (default port: 8321).
    
    For detailed llama-stack server setup instructions, refer to:
    - [Remote Setup Guide](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/kubernetes)
    - [Local Setup Guide](https://github.com/opendatahub-io/llama-stack-demos/blob/main/local_setup_guide.md)

In [1]:
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client import LlamaStackClient
import sys
sys.path.append('..')  
from src.utils import step_printer
from termcolor import cprint

inference_model = "ibm-granite/granite-3.2-8b-instruct"
tavily_search_api_key = "your-tavily-api-key"  # Replace with your Tavily API key

remote = True # Use the `remote` variable to switching between a local development environment and a remote kubernetes cluster.
remote_llama_stack_endpoint = "your-llama-stack-endpoint"  # Replace with your llama-stack endpoint if remote is set to True
local_llama_stack_endpoint = "http://localhost:8321"  # or set it to default port 8321 if remote is set to False.

stream_output = False # Set to True to stream the output of the agent.

print(f"Model: {inference_model}")

Model: ibm-granite/granite-3.2-8b-instruct


#### 1.2 Client Initialization
- Initialize the `LlamaStackClient` with the appropriate base URL and provider data.  
- Connect the llama-stack client to the llama-stack server using the base URL.

In [2]:
base_url = remote_llama_stack_endpoint if remote else local_llama_stack_endpoint
client = LlamaStackClient(
        base_url=base_url,
        provider_data={"tavily_search_api_key": tavily_search_api_key}
)

### 2. Configuring an agent for tool use.
- **Agent Initialization**: 

- Create an `Agent` instance with the desired model, instructions and tools."

- **Instructions**: The `instructions` parameter, also referred to as the system prompt, specifies the agent's role and behavior. In this example, the agent is configured as a helpful web search assistant. It is instructed to use a tool whenever a web search is required and to respond in a friendly and helpful tone.

- **Tools**: The `tools` parameter defines the tools available to the agent. In this case, the `builtin::websearch` tool is used, which enables the agent to perform web searches. This tool is essential for retrieving up-to-date information from the web. Internally, it leverages Tavily Search to execute the search queries efficiently.

- **How It Works**: When a user query is provided, the agent processes the input and determines whether a tool is required to fulfill the request. If the query involves retrieving information from the web, the agent invokes the `builtin::websearch` tool. The tool interacts with Tavily Search to fetch real-time data, which is then processed and returned to the user in a friendly and helpful tone. This workflow ensures that the agent can handle a wide range of queries effectively.

For more details on the `builtin::websearch` tool and its capabilities, refer to the [Llama-stack tools documentation](https://llama-stack.readthedocs.io/en/latest/building_applications/tools.html#web-search-providers). 

In [3]:
agent = Agent(
    client, 
    model=inference_model,
    instructions="""You are a helpful websearch assistant. When you are asked to search the web you must use a tool. 
            Whenever a tool is called, be sure return the response in a friendly and helpful tone.
            """ ,
    tools=["builtin::websearch"],
    sampling_params={"max_tokens":4096}
)

### 3. Running the agent.
- Define user prompts to interact with the agent.
- Use the agent to create a session and process user queries.
- Display the agent's responses for each query.

In [4]:
user_prompts = [
    "What’s latest in OpenShift?",
    "Search for recent news articles about advancements in quantum computing.",
    "Find coffee shops near Dublin city center that are open now, have Wi-Fi.",
]
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    session_id = agent.create_session("web-session")
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream_output
    )
    if stream_output:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


Processing user query: What’s latest in OpenShift?

---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: brave_search, Arguments: {'query': 'latest updates on OpenShift'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'query': 'latest updates on OpenShift',
│   'top_k': [
│   │   {
│   │   │   'title': 'Azure Red Hat OpenShift: April 2025 Update | Microsoft Community Hub',
│   │   │   'url': 'https://techcommunity.microsoft.com/blog/appsonazureblog/azure-red-hat-openshift-april-2025-update/4400331',
│   │   │   'content': "Azure Red Hat OpenShift's April update boosts security with Managed Identity, expands global reach, and enhances the platform for enterprise Kubernetes. ... OpenShift 4.16 on Day 0 → Now available for new ARO cluster installs with the latest features and improvements.",
│   │   │   'score': 0.8377398,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Experience the many enhancements in OpenShift 4.18',
│   │   │   'url': 'https://developers.redhat.com/blog/2025/02/25/whats-new-developers-red-hat-openshift-418',
│   │   │   'content': 'Experience the many enhancements in OpenShift 4.18 | Red Hat Developer Red Hat OpenShift Red Hat OpenShift Try Red Hat products and technologies without setup or configuration fees for 30 days with this shared Openshift and Kubernetes cluster. Red Hat OpenShift We are excited to highlight the new updates and improvements for developers in the Red Hat OpenShift 4.18 release. Red Hat Developer Hub, based on the\xa0Backstage project, provides software templates and plugins for OpenShift deployments, access to pipeline runs, viewing clusters from OCM, and more. OpenShift Pipelines provide an integrated user experience with the\xa0developer perspective of the Red Hat OpenShift Container Platform web console. Red Hat OpenShift',
│   │   │   'score': 0.76701033,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift 4.17: What you need to know',
│   │   │   'url': 'https://www.redhat.com/en/blog/what-you-need-to-know-red-hat-openshift-417',
│   │   │   'content': 'Based on\xa0Kubernetes 1.30 and\xa0CRI-O 1.30,\xa0 OpenShift 4.17 features expanded control plane options, increased flexibility for virtualization and networking, new capabilities to leverage generative AI, and continued investment in\xa0Red Hat OpenShift Platform Plus. Red Hat OpenShift Platform Plus (OPP) is an expanded version of Red Hat OpenShift that provides a comprehensive hybrid cloud platform with built-in security features for enterprises to build, deploy, run, manage, and automate applications at scale. For\xa0Red Hat OpenShift Data Foundation (ODF), we’re promoting to general availability the use of customer-managed ODF with\xa0Red Hat OpenShift Service on AWS with hosted control planes clusters in ODF 4.17.z. Ju Lim works on the core Red Hat OpenShift Container Platform for hybrid and multi-cloud environments to enable customers to run Red Hat OpenShift anywhere.',
│   │   │   'score': 0.6633424,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Red Hat OpenShift Container Platform Life Cycle Policy',
│   │   │   'url': 'https://access.redhat.com/support/policy/updates/openshift/',
│   │   │   'content': 'Extended Update Support - Long Life Add-on - Additional Term 1 enables customers to remain with the same minor release of Red Hat OpenShift for a total 24 months, allowing for stable production environments for mission-critical applications. Extended Update Support - Long Life Add-on - Additional Term 1 is provided with all Premium subscriptions for x86-64 versions of Red Hat OpenShift Kubernetes Engine, Red Hat OpenShift Container Platform, and Red Hat OpenShift Platform Plus. The combination of Extended Update Support - Long Life Add-On - Additional Term 1 and Extended Update Support Long Life Add-on - Additional Term 2 (footnote [13]) enables customers to remain with the same minor release of Red Hat OpenShift for a total of 36 months, allowing for stable production environments for mission-critical applications.',
│   │   │   'score': 0.47504577,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "What's 


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are the latest updates on OpenShift:

1. **Azure Red Hat OpenShift: April 2025 Update**
   - Boosts security with Managed Identity.
   - Expands global reach.
   - Enhances the platform for enterprise Kubernetes.
   - OpenShift 4.16 is now available for new ARO cluster installs with the latest features and improvements.

2. **OpenShift 4.18 Enhancements**
   - Red Hat Developer Hub, based on the Backstage project, provides software templates and plugins for OpenShift deployments, access to pipeline runs, viewing clusters from OCM, and more.
   - OpenShift Pipelines provide an integrated user experience with the developer perspective of the Red Hat OpenShift Container Platform web console.

3. **Red Hat OpenShift 4.17: What you need to know**
   - Based on Kubernetes 1.30 and CRI-O 1.30.
   - Features expanded control plane options, increased flexibility for virtualization and networking, and new capabilities to lever

{
│   'query': 'recent news articles about advancements in quantum computing',
│   'top_k': [
│   │   {
│   │   │   'title': 'Quantum Computing News -- ScienceDaily',
│   │   │   'url': 'https://www.sciencedaily.com/news/matter_energy/quantum_computing/',
│   │   │   'content': "Quantum Computing News March 2, 2025 Top Headlines Quantum Interference in Molecule-Surface Collisions Feb. 27, 2025 — Scientists have revealed how quantum interference and symmetry dictate molecular behavior in collisions with gold surfaces, offering new insights into molecular interactions. Unraveling How a 'Magnetic Twist' Induces One-Way Electric Flow Feb. 27, 2025 — A twist you'll never see coming: a breakthrough in understanding the relationship between chirality and electric flow at a microscopic level may help us develop chiral information ... Materials Incorporated Into Quantum Qubit Platform Feb. 25, 2025 — Researchers detail advances in the measurement of quantum devices that will be needed to realize a topological quantum computer. The ... New Low-Cost Challenger to Quantum Computer: Ising Machine Feb. 26, 2025 — A low-energy challenger to the quantum computer also works at room temperature. Quantum sensors ... Topological Quantum Processor Marks Breakthrough in Computing Feb. 20, 2025 — In a leap forward for quantum computing, physicists unveiled an eight-qubit topological quantum processor, the first of its kind.",
│   │   │   'score': 0.8815438,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Recent Breakthroughs Accelerate The Race For Quantum Computing - Forbes',
│   │   │   'url': 'https://www.forbes.com/sites/drektadang/2025/03/09/recent-breakthroughs-accelerate-the-race-for-quantum-computing/',
│   │   │   'content': "The race toward scalable quantum computing has reached a pivotal moment, with major players like Microsoft, Google, and IBM pushing forward with breakthroughs. Microsoft's recent announcement of its Majorana 1 chip marks a significant milestone, while Google’s Willow chip and IBM’s long-term quantum roadmap illustrate the industry’s diverse approaches to achieving fault-tolerant quantum systems. Unlike conventional qubit technologies, which require extensive error correction, Microsoft’s approach aims to build fault tolerance directly into the hardware, significantly improving the feasibility of large-scale quantum computing. While Microsoft, Google, and IBM push forward, Nvidia CEO Jensen Huang remains skeptical, stating at CES 2025 that quantum computing’s practical use may still be 20 years away.",
│   │   │   'score': 0.86331123,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Quantum computing | MIT News | Massachusetts Institute of Technology',
│   │   │   'url': 'https://news.mit.edu/topic/quantum-computing',
│   │   │   'content': 'February 5, 2025 Read full story → Fast control methods enable record-setting fidelity in superconducting qubit The advance holds the promise to reduce error-correction resource overhead. January 13, 2025 Read full story → MIT physicists predict exotic form of matter with potential for quantum computing New work suggests the ability to create fractionalized electrons known as non-Abelian anyons without a magnetic field, opening new possibilities for basic research and future applications. October 30, 2024 Read full story → Toward a code-breaking quantum computer Building on a landmark algorithm, researchers propose a way to make a smaller and more noise-tolerant quantum factoring circuit for cryptography. July 31, 2024 Read full story → Modular, scalable hardware architecture for a quantum computer A new quantum-system-on-chip enables the efficient control of a large array of qubits, moving toward practical quantum computing. May 2, 2024 Read full story → MIT scientists tune the entanglement structure in an array of qubits The advance offers a way to characterize a fundamental resource needed for quantum computing.',
│   │   │   '


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some recent news articles about advancements in quantum computing:

1. **Quantum Computing News -- ScienceDaily**
   - Title: "Quantum Computing News March 2, 2025 Top Headlines"
   - Content: This article covers several recent breakthroughs in quantum computing, including quantum interference in molecule-surface collisions, understanding the relationship between chirality and electric flow, and the development of a topological quantum processor.
   - [Read Full Story](https://www.sciencedaily.com/news/matter_energy/quantum_computing/)

2. **Recent Breakthroughs Accelerate The Race For Quantum Computing - Forbes**
   - Title: "Recent Breakthroughs Accelerate The Race For Quantum Computing"
   - Content: This piece discusses the rapid progress in the field, with major companies like Microsoft, Google, and IBM making significant strides. Microsoft's Majorana 1 chip, Google's Willow chip, and IBM's long-term quantum

{
│   'query': 'coffee shops near Dublin city center open now with Wi-Fi',
│   'top_k': [
│   │   {
│   │   │   'title': 'Top 10 Best Coffee & Cafes Near Dublin, Dublin - Yelp',
│   │   │   'url': 'https://www.yelp.com/search?find_desc=Coffee+&+Cafes&find_loc=Dublin',
│   │   │   'content': 'Open Now --:--Offers Delivery ... Top 10 Best Coffee & Cafes Near Dublin, Dublin. Sort: Recommended. All. Price. Open Now Offers Delivery Offers Takeout Free Wi-Fi Outdoor Seating. 1. Keoghs Café. 4.0 (115 reviews) South Inner City €€ Closed until 6:30 AM "Such a handy little coffee shop to get to in the city centre!',
│   │   │   'score': 0.89849097,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': 'Best Free Wi-Fi Cafes in Dublin - UPDATED: March, 2025',
│   │   │   'url': 'https://laptopfriendlycafe.com/free-wifi-cafes-in-dublin',
│   │   │   'content': "For those craving a cozy spot to work or study while enjoying a cup of coffee, Dublin boasts a plethora of cafes with free wifi that cater to students, freelancers, and remote workers. Many cafes in Dublin not only offer free wifi, but also serve delicious pastries, sandwiches, and specialty coffees to keep you fueled and focused throughout the day. So grab your laptop, order a hot cup of coffee, and settle in for a productive day of work or study at one of Dublin's best cafes with free wifi. Experience the charming and bustling atmosphere of our cafe, perfect for a delightful meal with friends or a quick coffee break. This café is a delightful hidden gem, perfect for those seeking a cozy atmosphere combined with great coffee and fresh food.",
│   │   │   'score': 0.7476348,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '10 Best Places To Grab A Coffee And Work In Dublin With Free Wifi',
│   │   │   'url': 'https://lovindublin.com/dublin/best-places-work-remotely-dublin-free-wifi',
│   │   │   'content': "In terms of trendy spost that fit in with the tech culture of local internet giants like Google and Facebook this is the perfect spot. The industrial design and award winning coffee will keep you coming back here time and again. Coffee is very subjective but I'd say this is the best in Dublin. Full map (Google Maps link) Starbucks",
│   │   │   'score': 0.6353361,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': "11 Brilliant Cafés And Coffee Shops in Dublin: A Local's Guide",
│   │   │   'url': 'https://www.timeout.com/dublin/restaurants/best-cafes-in-dublin',
│   │   │   'content': 'Dublin has chic coffee shops in spades (in fact it has so many great spots, choosing the best is not an easy task). For coffee, sweet treats and everything in between, here are the best cafés in Dublin. The best coffee shops in Dublin Brindle Coffee & Wine is one of the best coffee shops in the city for sitting down in the sunshine and watching the world go by along the terraced street. Dublin’s top contemporary menswear shop is also the best place in Temple Bar to grab a coffee. Serving up baked goods from No Messin’ Bakery, Proper Order is a firm favourite with Dublin’s coffee lovers. This local landmark in the cosy neighbourhood of Ranelagh is one of Dublin’s most beloved coffee shops.',
│   │   │   'score': 0.5402697,
│   │   │   'raw_content': None
│   │   },
│   │   {
│   │   │   'title': '10 Great Coffee Shops In Dublin That Are Perfect For Working In',
│   │   │   'url': 'https://lovindublin.com/food/10-great-coffee-shops-that-perfect-for-working-in',
│   │   │   'content': "A cosy little spot that serves fantastic coffee. Get upstairs if you can - it's a little more peaceful when you're away from the constant stream of people getting their takeaway coffee. 10. Coffee Works. This little spot is outside of the city centre, but the coffee is not to be missed.",
│   │   │   'score': 0.35830867,
│   │   │   'raw_content': None
│   │   }
│   ]
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Here are some coffee shops near Dublin city center that are currently open and offer Wi-Fi:

1. **Keoghs Café**
   - Address: South Inner City
   - Yelp Review: "Such a handy little coffee shop to get to in the city centre!"
   - Note: It's recommended to check their exact opening hours as they might vary.

2. **Cafes with Free Wi-Fi in Dublin**
   - Dublin is known for its numerous cafes offering free Wi-Fi, perfect for students, freelancers, and remote workers. They serve a variety of food and drinks to keep you energized.

3. **Industrial Design Coffee Shop**
   - This spot is popular among tech enthusiasts, serving award-winning coffee. It's a great place for a productive work session.

4. **Brindle Coffee & Wine**
   - Located in a terraced street, this café is perfect for people-watching while enjoying your coffee.

5. **Proper Order**
   - A favorite among Dublin's coffee lovers, it serves baked goods from No Messi

## Output Analysis
Here, we can observe that the `builtin::websearch` tool is used to perform a web search. The outputs are displayed in the notebook with color-coded text to help interpret the process:

- **Blue Text**: Represents the user's input or query.
- **Yellow Text**: Displays the LLM's inference response. 
- **Green Text**: Indicates the tool execution process, such as the tool being called and the query being sent to the web search API.

Great! 
We can see that the model returned some relevant and recent information about OpenShift. This was only possible due to its ability to call tools like the web search, demonstrating the agent's capacity to retrieve real-time data effectively.

For the second query, the model fetched the latest news articles, including 2 from March 2025. This is particularly impressive since the Granite 3.2 8B model, released in February 2025, has a knowledge cutoff date of April 2024. This demonstrates the agent's capability to extend beyond its static knowledge base by dynamically retrieving current information.  

For the third query about nearby coffee shops, the agent effectively used the `builtin::websearch` tool to provide practical, location-specific results with valid coffee shops and links, highlighting its real-world applicability.  

## Key Takeaways
- This tutorial demonstrates how to set up and use a single tool with AI Agent.
- It highlights the flexibility of switching between remote and local environments.
- By following these steps, users can quickly get started with AI Agents and explore its capabilities.

For more advanced use cases, please check our other example jupyter notebooks.